## hindcast example

This is a hindcast workflow using D3D. It uses new data every 12h.

In [ ]:
#to use the full width of the browser window
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
from pyPoseidon.utils import cast, data, point
import pickle
import pandas as pd
import datetime

In [ ]:
pyPoseidon.__version__

### setup

In [ ]:
#read the info from the first run
with open('/Users/brey/Downloads/EUR/D3D/20100201.00/eur025_info.pkl', 'r') as f:
              info=pickle.load(f)

In [ ]:
#define some info
info.update({'path':'/Users/brey/Downloads/EUR/D3D/', # The path of the project
     'case':'European 2010', # a reference tag
     })

In [ ]:
#creating a time sequence of the runs
#creating a time sequence of the runs
start_date = pd.to_datetime('2010-2-1 0:0:0')
end_date = pd.to_datetime('2010-2-2 0:0:0')
date_list = pd.date_range(start_date,end_date, freq='12H')
date_list

In [ ]:
#append to dic
info.update({'start_date':start_date,'end_date':end_date, 'dates' : date_list})

In [ ]:
#creating a sequence of folder to store the runs. In this case we name them after the date attribute.
#NOTE that the first folder is the fisrt run already perfomed!!
folders = [datetime.datetime.strftime(x, '%Y%m%d.%H') for x in date_list]
info.update({'folders':folders})

In [ ]:
#creating a sequence of folder from which we read the meteo.
meteo = []
PATH='/Users/brey/Downloads/'
for date in date_list:
    end_date= pd.to_datetime(date) + pd.to_timedelta(info['time_frame'])
    end_date = end_date.strftime(format='%Y-%m-%d %H:%M:%S')
    dr = pd.date_range(date, end_date, freq='12H')
    dur = [PATH+'{:04d}/{:02d}/{:02d}/'.format(x.year,x.month,x.day)+datetime.datetime.strftime(x, '%Y%m%d.%H')+'.tropical_cyclone.grib' for x in dr]
    meteo.append(dur)

In [ ]:
info.update({'meteo_files':meteo})

In [ ]:
#check
info['folders'],info['meteo_files']

In [ ]:
#check dict
#info

### run the hindcast

In [ ]:
#set cast
h = cast.cast(**info) # initialize

In [ ]:
h.run()

## Analysis of data

In [ ]:
%matplotlib inline

In [ ]:
folders = [info['path']+f for f in info['folders']]
folders

In [ ]:
otp = data(folders=folders,solver='d3d')#,sa_date='2010-2-1',se_date='2010-2-2 12:0:0') # optionally set range for tide gauge data acquisition 

In [ ]:
otp.impl.dem.bathymetry.plot()

In [ ]:
otp.impl.Dataset.S1[10,:,:].plot()

In [ ]:
otp.impl.Dataset.S1[:,23,30].plot.line()

In [ ]:
otp.impl.obs.locations

In [ ]:
# Get with index number
p = otp.impl.obs.iloc(2)
print p.head()
print p.tail()

In [ ]:
plat,plon = otp.impl.obs.locations.loc[2,['lat','lon']]

In [ ]:
ts = point(lon=plon,lat=plat,data=otp.impl)
ts.tseries(var='S1')

In [ ]:
## Join the graphs
ax = p.plot(figsize=(14,10))
ts.S1.plot(ax=ax,color=['r'],label='d3d')
ax.legend(loc='center left',bbox_to_anchor=(1.1, 0.5))